<a href="https://colab.research.google.com/github/MTBXC/MTBXC/blob/main/Automatic_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install python-dotenv
import kagglehub
import os
from dotenv import load_dotenv

# Zdefiniuj ścieżkę do pliku .env na Twoim Google Drive
dotenv_path = '/content/drive/MyDrive/API_KEY/kaggle.env'

# Sprawdź, czy plik istnieje przed próbą jego wczytania
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)

    # Pobierz zmienne środowiskowe
    kaggle_login = os.getenv('KAGGLE_LOGIN')
    kaggle_key = os.getenv('KAGGLE_KEY')

    # Możesz teraz użyć kaggle_login i kaggle_key
    print(f"KAGGLE_LOGIN: {kaggle_login}")
    print(f"KAGGLE_KEY: {kaggle_key}")
else:
    print(f"Plik .env nie został znaleziony w {dotenv_path}")




Mounted at /content/drive
KAGGLE_LOGIN: michakorlatowicz
KAGGLE_KEY: a5076292f15fe8110932d649a146f5ca


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

print('Importing data sources...')

titanic_path = kagglehub.competition_download('titanic')

print('Data source import complete.')


Importing data sources...


UnauthenticatedError: User is not authenticated